In [31]:
#import needed libs
import numpy as np
import torchmetrics
import pandas as pd
import csv
import torch
import torch.nn as nn
import random
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import argparse
from sklearn.metrics import f1_score, confusion_matrix

In [153]:
#define the arguments, so that control over the pipeline will be convinient
parser = argparse.ArgumentParser()
parser.add_argument('--batch_size', type=int, default=300)
parser.add_argument('--train_epoch', type=int, default=20)
parser.add_argument('--lr', type=float, default=1e-3)
parser.add_argument('--verbose', type=int, default=10)
parser.add_argument('--data_path', type=str, default="filtered_df2019.csv") #data path
parser.add_argument('--device', type=str, default='cuda:0')
parser.add_argument('--latent_dim', type=int, default=64)
parser.add_argument('--input_length', type=int, default=20)
parser.add_argument('--class_num', type=int, default=3)
parser.add_argument('--EQL', type=int, default=1024)   #LLM output dimension
parser.add_argument('--eql', type=int, default=5)
args = parser.parse_known_args()[0]

In [154]:
#this method combines the numerical features with the categorical features
def formator(line):
    result = []
    result.append(to_float(line['satv_use']))
    result.append(to_float(line['satm_use']))
    result.append(to_float(line['satcomp_use']))
    result.append(to_float(line['act_eng']))
    result.append(to_float(line['act_math']))
    result.append(to_float(line['act_read']))
    result.append(to_float(line['act_sci']))
    result.append(to_float(line['act_comp']))
    result.append(to_float(line['nc1']))
    result.append(to_float(line['nc2']))
    result.append(to_float(line['nc3']))
    result.append(to_float(line['ncav']))
    result.append(to_float(line['hs_gpa']))   
    result.append(to_float(line['college_gpa']))
    label = [0,0,0]
    if line['vt_adm_dec'][:2] == 'Ad':
        label[0] = 1
    elif line['vt_adm_dec'][:2] == 'De':
        label[1] = 1
    else:
        label[2] = 1
    #coll(result,line)
    #gender(result,line)
    ethnic(result,line)
    return result, label

#below are some helper functions to process the categorical features
def coll(result, line):
    college = [0 for _ in range(8)]
    strings = ['Eng', 'Col', 'Bus', 'Int', 'Lib', 'Agr', 'Arc', 'Nat']
    begin = line['vt_coll'][:3]
    if begin in strings:
        idx = strings.index(begin)
        college[idx] += 1
    result += college

def gender(result, line):
    gend = [0,0]
    if line['gender'][:2] == 'Ma':
        gend[0] += 1
    elif line['gender'][:2] == 'Fe':
        gend[1] += 1
    result += gend

def ethnic(result, line):
    ethn = [0 for _ in range(6)]
    strings = ['Whi', 'Asi', 'His', 'Non', 'Bla', 'Two']
    begin = line['ethnic/race'][:3]
    if begin in strings:
        idx = strings.index(begin)
        ethn[idx] += 1
    result += ethn

def to_float(input):
    if input == '':
        return -1
    else:
        return float(input)

In [155]:
# Load previously saved textual embeddings
EQ_data = pd.read_pickle("data2.pkl")

In [158]:
#read input data from file, and save them into a certain format
with open(args.data_path, 'r', newline='', encoding='utf-8') as file:
    data = []
    label = []
    reader = csv.DictReader(file)
    for row in reader:
        features, decision = formator(row)
        data.append(features)
        label.append(decision)
    data = np.array(data)
    data = (data - np.min(data,axis=0))/(np.max(data,axis=0) - np.min(data,axis=0) + 1e-6)
    label = np.array(label,dtype=float)

EQ1_array = np.array(EQ_data['EQ1_embeddings'].tolist())
EQ2_array = np.array(EQ_data['EQ2_embeddings'].tolist())
EQ3_array = np.array(EQ_data['EQ3_embeddings'].tolist())
EQ4_array = np.array(EQ_data['EQ4_embeddings'].tolist())

min_rows = min(data.shape[0], EQ1_array.shape[0], EQ2_array.shape[0], EQ3_array.shape[0], EQ4_array.shape[0])

# Truncate all to the smallest size
data = data[:min_rows, :]
EQ1_array = EQ1_array[:min_rows, :]
EQ2_array = EQ2_array[:min_rows, :]
EQ3_array = EQ3_array[:min_rows, :]
EQ4_array = EQ4_array[:min_rows, :]

# Concatenate all embeddings
data = np.concatenate([data, EQ1_array, EQ2_array, EQ3_array, EQ4_array], axis=-1)

In [159]:
# Initialize the lists for storing new data and labels
new_data = []
new_label = []

# Initialize counters for each label category
numbers = [0, 0, 0]

# Iterate over the labels
for i in range(label.shape[0]):
    for j in range(3):
        # Check if the current label is 1 and the counter is below 4000
        if label[i][j] == 1 and numbers[j] < 4000:
            numbers[j] += 1  # Increment the counter
            new_data.append(data[i])  # Append the data
            new_label.append(label[i])  # Append the label

# Convert the collected data and labels to tensors
data = torch.tensor(np.array(new_data))
label = torch.tensor(np.array(new_label))

# Print the resulting label shape and counters
print("Label shape:", label.shape)
print("Counters:", numbers)

Label shape: torch.Size([12000, 3])
Counters: [4000, 4000, 4000]


In [161]:
#create train, val, test data and labels
idx = list(range(data.shape[0]))
idx = np.linspace(0,data.shape[0]-1,data.shape[0],dtype=int)
random.shuffle(idx)
train_size = int(data.shape[0]*0.7)
val_size = int(data.shape[0]*0.15)
train_idx = idx[:train_size]
val_idx = idx[train_size:train_size+val_size]
test_idx = idx[train_size+val_size:]
train_data = data[train_idx]
val_data = data[val_idx]
test_data = data[test_idx]
train_label = label[train_idx]
val_label = label[val_idx]
test_label = label[test_idx]

In [162]:
#define the dataset class
class Admission_Dataset(Dataset):
    def __init__(self, data, label):
        self.data = data
        self.label = label
    def __getitem__(self, index):
        return self.data[index], self.label[index]
    def __len__(self):
        return self.data.shape[0]

#create dataset and data loader
train_dataset = Admission_Dataset(train_data,train_label)
val_dataset = Admission_Dataset(val_data,val_label)
test_dataset = Admission_Dataset(test_data,test_label)
train_loader = DataLoader(dataset=train_dataset, batch_size=args.batch_size, shuffle=False)
val_loader = DataLoader(dataset=val_dataset, batch_size=args.batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=args.batch_size, shuffle=False)

In [164]:
#define the training pipeline
def train(model, args):
    # Define optimizer and loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
    criterion = nn.CrossEntropyLoss()
    #criterion = F1Score(num_classes=3, average='macro').to(args.device)

    # Training loop
    for epoch in tqdm(range(args.train_epoch)):
        model.train()  # Set model to training mode
        for batch_idx, batch in enumerate(train_loader):
            data, label = batch
            # Move data and labels to the same device as the model
            device = next(model.parameters()).device
            data = data.to(device)
            label = label.to(device)
            
            # Forward pass
            preds = model(data.float())
            
            # Backward pass
            optimizer.zero_grad()
            loss = criterion(preds, label)
            loss.backward()
            optimizer.step()
            
            # Print loss for the batch
            if batch_idx % args.verbose == 0:
                _, f1, acc = test(model, 'val')
                print(f'Train Epoch {epoch}, Batch {batch_idx}, Loss: {loss.item():.4f}, F1: {f1.item():.4f}, acc: {acc.item():.4f}')
        
        # Evaluate and print classification metrics for the epoch
        class_accuracies, overall_f1, overall_acc = test(model, 'test')
        
        # Print metrics
        print(f'\nTrain Epoch {epoch} Metrics:')
        for i, acc in enumerate(class_accuracies):
            print(f'Class {i} Accuracy: {acc:.4f}')
        print(f'Overall F1 Score: {overall_f1:.4f}\n')
        print(f'Overall accuracy: {overall_acc:.4f}\n')

#this method can be used to validate the model or test the model, the parameter "mode" can switch whether to validate or test
def test(model, mode='test'):
    model.eval()  # Set model to evaluation mode
    all_preds = []
    all_labels = []
    if mode == 'test':
        loader = test_loader
    else:
        loader = val_loader
    with torch.no_grad():  # Disable gradient calculations
        for batch in loader:
            data, label = batch
            # Move data and label to the same device as the model
            device = next(model.parameters()).device
            data = data.to(device)
            label = label.to(device)

            # Forward pass
            preds = model(data.float())
            preds = torch.argmax(preds, dim=1)
            label = torch.argmax(label, dim=1)

            # Collect predictions and labels
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(label.cpu().numpy())

    # Calculate overall F1 score
    overall_f1 = f1_score(all_labels, all_preds, average='macro')
    print(torch.tensor(all_labels[1]))
    print(torch.tensor(all_preds[1]))
    print(len(all_labels))
    overall_acc = torch.sum(torch.tensor(all_labels) == torch.tensor(all_preds))/len(all_labels)
    # Confusion matrix to calculate per-class accuracy
    cm = confusion_matrix(all_labels, all_preds)
    class_accuracies = cm.diagonal() / cm.sum(axis=1)
    
    return class_accuracies, overall_f1, overall_acc


In [165]:
#this class defines the backbone model, which is an MLP
class MLP(torch.nn.Module):
    def __init__(self, latent_dim = 64, input_length=12, class_num=3, EQL=1024, eql=5):
        super().__init__()
        self.input_length=input_length
        self.lin1 = nn.Linear(input_length+4*eql,latent_dim)
        self.lin2 = nn.Linear(latent_dim,latent_dim)
        #self.lin21 = nn.Linear(latent_dim,latent_dim)
        self.lin3 = nn.Linear(latent_dim,class_num)
        self.EQ = nn.Linear(EQL,eql)
        self.EQL = EQL
        self.eql = eql

    def forward(self,x):
        x0 = x[:,:self.input_length]
        EQ1 = self.EQ(x[:,self.input_length:self.input_length+self.EQL])
        EQ2 = self.EQ(x[:,self.input_length+self.EQL:self.input_length+2*self.EQL])
        EQ3 = self.EQ(x[:,self.input_length+2*self.EQL:self.input_length+3*self.EQL])
        EQ4 = self.EQ(x[:,self.input_length+3*self.EQL:])
        x = torch.cat([x0,EQ1,EQ2,EQ3,EQ4],dim=-1)
        x = self.lin1(x)
        x = torch.nn.GELU()(x)
        x = self.lin2(x)
        x = torch.nn.GELU()(x)
        ##x = self.lin21(x)
        ##x = torch.nn.GELU()(x)
        x = self.lin3(x)
        return x
    
    def save_rep(self, x):
        x0 = x[:,:self.input_length]
        EQ1 = self.EQ(x[:,self.input_length:self.input_length+self.EQL])
        EQ2 = self.EQ(x[:,self.input_length+self.EQL:self.input_length+2*self.EQL])
        EQ3 = self.EQ(x[:,self.input_length+2*self.EQL:self.input_length+3*self.EQL])
        EQ4 = self.EQ(x[:,self.input_length+3*self.EQL:])
        x = torch.cat([x0,EQ1,EQ2,EQ3,EQ4],dim=-1)
        np.save('original_rep.npy', x.detach().cpu().numpy())
        x = self.lin1(x)
        np.save('layer1_rep.npy', x.detach().cpu().numpy())
        x = torch.nn.GELU()(x)
        x = self.lin2(x)
        np.save('layer2_rep.npy', x.detach().cpu().numpy())

In [166]:
#initialize a model, and train it, validation is done simultaneously during the training process
model = MLP(latent_dim=args.latent_dim,
            input_length=args.input_length,
            class_num=args.class_num,
            EQL=args.EQL,
            eql=args.eql
            ).to(args.device)
train(model, args)

  0%|          | 0/20 [00:00<?, ?it/s]

tensor(1)
tensor(1)
1800
Train Epoch 0, Batch 0, Loss: 1.1014, F1: 0.1807, acc: 0.3372
tensor(1)
tensor(0)
1800
Train Epoch 0, Batch 10, Loss: 1.0782, F1: 0.4106, acc: 0.4706
tensor(1)
tensor(0)
1800
Train Epoch 0, Batch 20, Loss: 1.0194, F1: 0.4305, acc: 0.5250


  5%|▌         | 1/20 [00:00<00:05,  3.32it/s]

tensor(1)
tensor(2)
1800

Train Epoch 0 Metrics:
Class 0 Accuracy: 0.6869
Class 1 Accuracy: 0.3328
Class 2 Accuracy: 0.5128
Overall F1 Score: 0.5015

Overall accuracy: 0.5083

tensor(1)
tensor(2)
1800
Train Epoch 1, Batch 0, Loss: 0.9733, F1: 0.5427, acc: 0.5428
tensor(1)
tensor(2)
1800
Train Epoch 1, Batch 10, Loss: 0.9179, F1: 0.5642, acc: 0.5650


 10%|█         | 2/20 [00:00<00:05,  3.32it/s]

...

tensor(1)
tensor(2)
1800
Train Epoch 17, Batch 20, Loss: 0.7068, F1: 0.5855, acc: 0.5828
tensor(1)
tensor(1)
1800

Train Epoch 17 Metrics:
Class 0 Accuracy: 0.7441
Class 1 Accuracy: 0.6333
Class 2 Accuracy: 0.3492
Overall F1 Score: 0.5673

Overall accuracy: 0.5772

tensor(1)
tensor(2)
1800
Train Epoch 18, Batch 0, Loss: 0.6257, F1: 0.5955, acc: 0.6017


 95%|█████████▌| 19/20 [00:08<00:00,  2.92it/s]

tensor(1)
tensor(2)
1800
Train Epoch 18, Batch 10, Loss: 0.6838, F1: 0.5945, acc: 0.5961
tensor(1)
tensor(2)
1800
Train Epoch 18, Batch 20, Loss: 0.7054, F1: 0.5869, acc: 0.5850
tensor(1)
tensor(1)
1800

Train Epoch 18 Metrics:
Class 0 Accuracy: 0.7475
Class 1 Accuracy: 0.6349
Class 2 Accuracy: 0.3441
Overall F1 Score: 0.5665

Overall accuracy: 0.5772

tensor(1)
tensor(2)
1800
Train Epoch 19, Batch 0, Loss: 0.6181, F1: 0.5927, acc: 0.5989
tensor(1)
tensor(2)
1800
Train Epoch 19, Batch 10, Loss: 0.6788, F1: 0.5942, acc: 0.5961
tensor(1)
tensor(2)
1800
Train Epoch 19, Batch 20, Loss: 0.7036, F1: 0.5875, acc: 0.5867


100%|██████████| 20/20 [00:08<00:00,  2.36it/s]

tensor(1)
tensor(1)
1800

Train Epoch 19 Metrics:
Class 0 Accuracy: 0.7475
Class 1 Accuracy: 0.6317
Class 2 Accuracy: 0.3458
Overall F1 Score: 0.5657

Overall accuracy: 0.5767

